In [1]:
import spacy
from spacy import displacy
import json
!python3 -m spacy download en
import en_core_web_sm

nlp = en_core_web_sm.load()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/miniconda3/lib/python3.8/site-packages/en_core_web_sm -->
/opt/miniconda3/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

***
***CONSTANTS***
***

In [3]:
COPULAS = ["is", "are", "were"]
VERB_TAGS = ["AUX","VERB"]

In [4]:
sentence_group_1 = ["The whale is an animal.",
"The blue whale is an animal.",
"The whale is a quadrupedal mammal.",
"The chameleon is a polychromatic arboreal insectivore.",
"Carnivorous spiders are common.",
"All cephalopods are nocturnal.",
"The majority of cephalopods are nocturnal.",
"Some cephalopods are nocturnal.",
"No cephalopods are nocturnal.",
"Most terrestrial gastropods are hermaphrodites.",
"The simplest gastropods are the limpets and abalone.",
"A cephalopod is any member of the molluscan class Cephalopoda such as a squid, octopus, cuttlefish, or nautilus.",
"Gastropods were described as gastropodes by Georges Cuiver.",
"Cephalopods are widely regarded as the most intelligent of the invertebrates."]

sentence_group_2 = ["The fastest biped is the ostrich.",
"Not all tetrapods are quadrupeds and not all quadrupeds are tetrapods.",
"A tripod stance is a behaviour in which quadruped animals rear up on their hind legs and use their tail to support their weight.",
"Brontosaurus (meaning \"thunder lizard\" from Greek βροντή, \"thunder\" and σαῦρος, \"lizard\") is a genus of gigantic quadruped sauropod dinosaurs.",
"Conrad Gessner (1516–1565) was a Swiss naturalist and bibliographer and one of founding figures of modern zoology.",
"Sound waves with frequency above 20 kHz are called ultrasonic waves and are inaudble to humans, but some animals, like bats and dolphins, use them.",
"A timer is a kind of clock for measuring time intervals, often with an audible alarm that the time is up.",
"There is a debate among audiophiles on whether there is an audible difference between tubes and transistors in sound equipment.",
"A Geiger counter is an instrument that measures ionizing radiation such as alpha particles, beta particles, or gamma rays. The radiation sensor produces a series of audible clicks when radiation is present.",
"Autophagy is the process that cells use to break down and recycle cellular components."]

***
***HELPERS***
***

In [5]:
def visualize(doc):
    # https://spacy.io/api/top-level#displacy_options
    options= {
        "compact":True,
        "distance":100,
    }
    displacy.render(doc, style='dep',jupyter=True,options=options)    
    
def describe(sentence):
    doc = nlp((sentence))
    visualize(doc)
    print(json.dumps(doc.to_json(), indent=1))    
    
def first_or_none(lst):
    return None if len(lst) == 0 else lst[0]    

def flatten(lst):
    return [item for sublist in lst for item in sublist]

***
***DEPENDENCY TREE PARSING METHODS***
***

In [6]:
def children_for(json_doc, head_id, recursive=False):
    """Recursively find the children and the children's children for a word in a dependency tree.
    
    
    """    
    results = []
    
    for x in json_doc["tokens"]:
        if x["head"] == head_id and x["id"] != head_id:
            results.append(x)
            
    if recursive and len(results) > 0:
        return results + flatten([children_for(json_doc, r["id"], True) for r in results])
    else:
        return results

def find_token_by_attr(tokens, attr, value):
    results = [t for t in tokens if t[attr] == value]
    return first_or_none(results)

def filter_tokens_by_attr(tokens, attr, value):
    return [t for t in tokens if t[attr] != value]

def filter_tokens_by_attrs(tokens, k_v_pairs):
    for key, value in k_v_pairs:
        tokens = filter_tokens_by_attr(tokens, key, value)
    return tokens

def select_tokens_by_attr(tokens, attr, value):
    return [t for t in tokens if t[attr] == value]

def select_tokens_by_attrs(tokens, k_v_pairs):
    result = []
    for key, value in k_v_pairs:
        result += select_tokens_by_attr(tokens, key, value)
    return result

def get_verbs(json_doc, only=None):
    results = []
    for x in json_doc["tokens"]:
        value = value_for_token(x, json_doc)
        is_verb = x["pos"] in VERB_TAGS
        include = only is None or value in only
        if is_verb and include:
            results.append(x)
    return results

def sorted_values(tokens, json_doc):
    return [value_for_token(t, json_doc) for t in sorted(tokens, key=lambda w: w["id"])]

def value_for_token(token, json_doc):
    return json_doc["text"][token["start"]:token["end"]].lower()

In [13]:
VALID_PREDICATE_K_V_PAIRS = [
    ("dep","acomp"),
    ("dep","amod"),
    ("dep","attr")
]

# primary entry point
def mine_json_doc(json_doc, verbs):
    results = []
    
    for verb in get_verbs(json_doc, verbs):
        children = children_for(json_doc, verb["id"])
        print(f"parsing verb {value_for_token(verb, json_doc)}...")
        subject = find_token_by_attr(children, "dep", "nsubj")     
        if subject is None:
            continue
            
        print(f"\n\tsubject is: {value_for_token(subject, json_doc)}")
        predicates = select_tokens_by_attrs(children, VALID_PREDICATE_K_V_PAIRS)
                    
        subject_deps = children_for(json_doc, subject["id"])
        subject_deps = filter_tokens_by_attrs(subject_deps, [("pos", "DET"),("pos", "PUNCT")])
        print(f"\t\tdependencies are: {', '.join([value_for_token(o, json_doc) for o in subject_deps])}")
        word = [subject] + subject_deps
        
        properties = []
        
        print(f"\n\tpredicates are: {', '.join([value_for_token(o, json_doc) for o in predicates])}")
        
        for o in predicates:
            print(f"\n\tparsing predicate {value_for_token(o, json_doc)}...")
            properties.append(o)            
            other_deps = children_for(json_doc, o["id"], True)
            filtered = filter_tokens_by_attrs(other_deps, [("pos", "DET")])
            print(f"\t\tdependencies are: {', '.join([value_for_token(o, json_doc) for o in filtered])}")
            properties += filtered
            
        word = sorted_values(word, json_doc)
        properties = sorted_values(properties, json_doc)
            
        results.append((" ".join(word), " ".join(properties)))
        
    return results

def mine_sentence(sentence):
    return mine_json_doc(nlp((sentence)).to_json(), COPULAS)

***
***TEST***
***

In [14]:
s = "Gastropods were described as gastropodes by Georges Cuiver."

In [15]:
print(mine_sentence(s))

parsing verb were...
[]


In [16]:
describe(s)

{
 "text": "Gastropods were described as gastropodes by Georges Cuiver.",
 "ents": [
  {
   "start": 44,
   "end": 58,
   "label": "PERSON"
  }
 ],
 "sents": [
  {
   "start": 0,
   "end": 59
  }
 ],
 "tokens": [
  {
   "id": 0,
   "start": 0,
   "end": 10,
   "pos": "NOUN",
   "tag": "NNS",
   "dep": "nsubjpass",
   "head": 2
  },
  {
   "id": 1,
   "start": 11,
   "end": 15,
   "pos": "VERB",
   "tag": "VBD",
   "dep": "auxpass",
   "head": 2
  },
  {
   "id": 2,
   "start": 16,
   "end": 25,
   "pos": "VERB",
   "tag": "VBN",
   "dep": "ROOT",
   "head": 2
  },
  {
   "id": 3,
   "start": 26,
   "end": 28,
   "pos": "ADP",
   "tag": "IN",
   "dep": "prep",
   "head": 2
  },
  {
   "id": 4,
   "start": 29,
   "end": 40,
   "pos": "NOUN",
   "tag": "NNS",
   "dep": "pobj",
   "head": 3
  },
  {
   "id": 5,
   "start": 41,
   "end": 43,
   "pos": "ADP",
   "tag": "IN",
   "dep": "agent",
   "head": 2
  },
  {
   "id": 6,
   "start": 44,
   "end": 51,
   "pos": "PROPN",
   "tag": "NNP",

In [9]:
for sentence in sentence_group_1:
    json_doc = nlp((sentence)).to_json()
    predictions = mine(json_doc, COPULAS)
    print(json_doc["text"], "\n", predictions, "\n")

The whale is an animal. 
 [('whale', 'animal')] 

The blue whale is an animal. 
 [('blue whale', 'animal')] 

The whale is a quadrupedal mammal. 
 [('whale', 'quadrupedal mammal')] 

The chameleon is a polychromatic arboreal insectivore. 
 [('chameleon', 'polychromatic arboreal insectivore')] 

Carnivorous spiders are common. 
 [('carnivorous spiders', 'common')] 

All cephalopods are nocturnal. 
 [('cephalopods', 'nocturnal')] 

The majority of cephalopods are nocturnal. 
 [('majority of', 'nocturnal')] 

Some cephalopods are nocturnal. 
 [('cephalopods', 'nocturnal')] 

No cephalopods are nocturnal. 
 [('cephalopods', 'nocturnal')] 

Most terrestrial gastropods are hermaphrodites. 
 [('most terrestrial gastropods', 'hermaphrodites')] 

The simplest gastropods are the limpets and abalone. 
 [('simplest gastropods', 'limpets and abalone')] 

A cephalopod is any member of the molluscan class Cephalopoda such as a squid, octopus, cuttlefish, or nautilus. 
 [('cephalopod', 'member of moll

In [94]:
spacy.explain("amod")

'adjectival modifier'

In [95]:
spacy.explain("acomp")

'adjectival complement'

In [11]:
'''
1. looking at every clause
2. looking if the verb matches a given verb or any of its other forms
3. taking the subject of that verbs and any of its children
4. also taking the predicated of that verb and any of its children
5. filter out things (like determiners, punctuation)
'''

'\n1. looking at every clause\n2. looking if the verb matches a given verb or any of its other forms\n3. taking the subject of that verbs and any of its children\n4. also taking the predicated of that verb and any of its children\n5. filter out things (like determiners, punctuation)\n'